# Demonstrate PyPSA unit commitment with a one-bus two-generator example


This tutorial runs through examples of unit commitment for generators at a single bus. Examples of minimum part-load, minimum up time, minimum down time, start up costs, shut down costs and ramp rate restrictions are shown.

To enable unit commitment on a generator, set its attribute committable = True.


Available as a Jupyter notebook at http://www.pypsa.org/examples/unit-commitment.ipynb.

In [1]:
import pypsa

## Minimum part load demonstration

In final hour load goes below part-load limit of coal gen (30%), forcing gas to commit.

In [2]:
nu = pypsa.Network()

nu.set_snapshots(range(4))

nu.add("Bus","bus")


nu.add("Generator","coal",bus="bus",
       committable=True,
       p_min_pu=0.3,
       marginal_cost=20,
       p_nom=10000)

nu.add("Generator","gas",bus="bus",
       committable=True,
       marginal_cost=70,
       p_min_pu=0.1,
       p_nom=1000)

nu.add("Load","load",bus="bus",p_set=[4000,6000,5000,800])

In [3]:
nu.lopf(nu.snapshots)

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 356000.0
  Upper bound: 356000.0
  Number of objectives: 1
  Number of constraints: 25
  Number of variables: 21
  Number of nonzeros: 45
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.024035930633544922
# ----------------------------------------------------------
#   Solution Information
# ---------------------------

('ok', 'optimal')

In [4]:
nu.generators_t.status

,coal,gas
0,1.0,0.0
1,1.0,0.0
2,1.0,0.0
3,0.0,1.0


In [5]:
nu.generators_t.p

,coal,gas
0,4000.0,-0.0
1,6000.0,-0.0
2,5000.0,-0.0
3,-0.0,800.0


## Minimum up time demonstration

Gas has minimum up time, forcing it to be online longer

In [6]:
nu = pypsa.Network()

nu.set_snapshots(range(4))

nu.add("Bus","bus")


nu.add("Generator","coal",bus="bus",
       committable=True,
       p_min_pu=0.3,
       marginal_cost=20,
       p_nom=10000)

nu.add("Generator","gas",bus="bus",
       committable=True,
       marginal_cost=70,
       p_min_pu=0.1,
       up_time_before=0,
       min_up_time=3,
       p_nom=1000)

nu.add("Load","load",bus="bus",p_set=[4000,800,5000,3000])

In [7]:
nu.lopf(nu.snapshots)

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk


gas up time before: 0 , must stay up: 0


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 306000.0
  Upper bound: 306000.0
  Number of objectives: 1
  Number of constraints: 28
  Number of variables: 21
  Number of nonzeros: 55
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.014981746673583984
# ----------------------------------------------------------
#   Solution Information
# ---------------------------

('ok', 'optimal')

In [8]:
nu.generators_t.status

,coal,gas
0,1.0,1.0
1,0.0,1.0
2,1.0,1.0
3,1.0,0.0


In [9]:
nu.generators_t.p

,coal,gas
0,3900.0,100.0
1,-0.0,800.0
2,4900.0,100.0
3,3000.0,-0.0


## Minimum down time demonstration

Coal has a minimum down time, forcing it to go off longer.

In [10]:
nu = pypsa.Network()

nu.set_snapshots(range(4))

nu.add("Bus","bus")


nu.add("Generator","coal",bus="bus",
       committable=True,
       p_min_pu=0.3,
       marginal_cost=20,
       min_down_time=2,
       down_time_before=1,
       p_nom=10000)

nu.add("Generator","gas",bus="bus",
       committable=True,
       marginal_cost=70,
       p_min_pu=0.1,
       p_nom=4000)

nu.add("Load","load",bus="bus",p_set=[3000,800,3000,8000])

In [11]:
nu.lopf(nu.snapshots)

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk


coal down time before: 1 , must stay down: 1


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 486000.0
  Upper bound: 486000.0
  Number of objectives: 1
  Number of constraints: 28
  Number of variables: 21
  Number of nonzeros: 52
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.01772904396057129
# ----------------------------------------------------------
#   Solution Information
# ----------------------------

('ok', 'optimal')

In [12]:
nu.objective

486000.0

In [13]:
nu.generators_t.status

,coal,gas
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0


In [14]:
nu.generators_t.p

,coal,gas
0,-0.0,3000.0
1,-0.0,800.0
2,3000.0,-0.0
3,8000.0,-0.0


## Start up and shut down costs

Now there are associated costs for shutting down, etc

In [15]:


nu = pypsa.Network()

nu.set_snapshots(range(4))

nu.add("Bus","bus")


nu.add("Generator","coal",bus="bus",
       committable=True,
       p_min_pu=0.3,
       marginal_cost=20,
       min_down_time=2,
       start_up_cost=5000,
       p_nom=10000)

nu.add("Generator","gas",bus="bus",
       committable=True,
       marginal_cost=70,
       p_min_pu=0.1,
       shut_down_cost=25,
       p_nom=4000)

nu.add("Load","load",bus="bus",p_set=[3000,800,3000,8000])

In [16]:
nu.lopf(nu.snapshots)

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk


coal down time before: 0 , must stay down: 0


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 491025.0
  Upper bound: 491025.0
  Number of objectives: 1
  Number of constraints: 36
  Number of variables: 29
  Number of nonzeros: 75
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.018759727478027344
# ----------------------------------------------------------
#   Solution Information
# ---------------------------

('ok', 'optimal')

In [17]:
nu.objective

491025.0

In [18]:
nu.generators_t.status

,coal,gas
0,0.0,1.0
1,0.0,1.0
2,1.0,0.0
3,1.0,0.0


In [19]:
nu.generators_t.p

,coal,gas
0,0.0,3000.0
1,-0.0,800.0
2,3000.0,-0.0
3,8000.0,-0.0


# Ramp rate limits

In [20]:
import pypsa

nu = pypsa.Network()

nu.set_snapshots(range(6))

nu.add("Bus","bus")


nu.add("Generator","coal",bus="bus",
       marginal_cost=20,
       ramp_limit_up=0.1,
       ramp_limit_down=0.2,
       p_nom=10000)

nu.add("Generator","gas",bus="bus",
       marginal_cost=70,
       p_nom=4000)

nu.add("Load","load",bus="bus",p_set=[4000,7000,7000,7000,7000,3000])

In [21]:
nu.lopf(nu.snapshots)

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 950000.0
  Upper bound: 950000.0
  Number of objectives: 1
  Number of constraints: 23
  Number of variables: 19
  Number of nonzeros: 39
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.01867985725402832
# ----------------------------------------------------------
#   Solution Information
# ----------------------------

('ok', 'optimal')

In [22]:
nu.generators_t.p

,coal,gas
0,4000.0,0.0
1,5000.0,2000.0
2,6000.0,1000.0
3,7000.0,0.0
4,5000.0,2000.0
5,3000.0,0.0


In [23]:
import pypsa

nu = pypsa.Network()

nu.set_snapshots(range(6))

nu.add("Bus","bus")


nu.add("Generator","coal",bus="bus",
       marginal_cost=20,
       ramp_limit_up=0.1,
       ramp_limit_down=0.2,
       p_nom_extendable=True,
       capital_cost=1e2)

nu.add("Generator","gas",bus="bus",
       marginal_cost=70,
       p_nom=4000)

nu.add("Load","load",bus="bus",p_set=[4000,7000,7000,7000,7000,3000])

In [24]:
nu.lopf(nu.snapshots)

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1675000.0
  Upper bound: 1675000.0
  Number of objectives: 1
  Number of constraints: 35
  Number of variables: 20
  Number of nonzeros: 67
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.020117759704589844
# ----------------------------------------------------------
#   Solution Information
# -------------------------

('ok', 'optimal')

In [25]:
nu.generators.p_nom_opt

coal    5000.0
gas     4000.0
Name: p_nom_opt, dtype: float64

In [26]:
nu.generators_t.p

,coal,gas
0,4000.0,0.0
1,4500.0,2500.0
2,5000.0,2000.0
3,5000.0,2000.0
4,4000.0,3000.0
5,3000.0,0.0


In [27]:
import pypsa

nu = pypsa.Network()

nu.set_snapshots(range(7))

nu.add("Bus","bus")


#Can get bad interactions if SU > RU and p_min_pu; similarly if SD > RD


nu.add("Generator","coal",bus="bus",
       marginal_cost=20,
       committable=True,
       p_min_pu=0.05,
       initial_status=0,
       ramp_limit_start_up=0.1,
       ramp_limit_up=0.2,
       ramp_limit_down=0.25,
       ramp_limit_shut_down=0.15,
       p_nom=10000.)

nu.add("Generator","gas",bus="bus",
       marginal_cost=70,
       p_nom=10000)

nu.add("Load","load",bus="bus",p_set=[0.,200.,7000,7000,7000,2000,0])

In [28]:
nu.lopf(nu.snapshots)

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk
INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1149000.0
  Upper bound: 1149000.0
  Number of objectives: 1
  Number of constraints: 41
  Number of variables: 29
  Number of nonzeros: 98
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.018232107162475586
# ----------------------------------------------------------
#   Solution Information
# -------------------------

('ok', 'optimal')

In [29]:
nu.generators_t.p

,coal,gas
0,-0.0,0.0
1,-0.0,200.0
2,1000.0,6000.0
3,3000.0,4000.0
4,4000.0,3000.0
5,1500.0,500.0
6,-0.0,0.0


In [30]:
nu.generators_t.status

,coal,gas
0,0.0,1.0
1,0.0,1.0
2,1.0,1.0
3,1.0,1.0
4,1.0,1.0
5,1.0,1.0
6,0.0,1.0


In [31]:
nu.generators.loc["coal"]

attribute
bus                       bus
control                 Slack
type                         
p_nom                   10000
p_nom_extendable        False
p_nom_min                   0
p_nom_max                 inf
p_min_pu                 0.05
p_max_pu                    1
p_set                       0
q_set                       0
sign                        1
carrier                      
marginal_cost              20
capital_cost                0
efficiency                  1
committable              True
start_up_cost               0
shut_down_cost              0
min_up_time                 0
min_down_time               0
up_time_before              1
down_time_before            0
ramp_limit_up             0.2
ramp_limit_down          0.25
ramp_limit_start_up       0.1
ramp_limit_shut_down     0.15
p_nom_opt               10000
Name: coal, dtype: object

# Rolling horizon example

This example solves sequentially in batches

In [32]:
sets_of_snapshots = 6
p_set = [4000,5000,700,800,4000]

nu = pypsa.Network()

nu.set_snapshots(range(len(p_set)*sets_of_snapshots))

nu.add("Bus","bus")


nu.add("Generator","coal",bus="bus",
       committable=True,
       p_min_pu=0.3,
       marginal_cost=20,
       min_down_time=2,
       min_up_time=3,
       up_time_before=1,
       ramp_limit_up=1,
       ramp_limit_down=1,
       ramp_limit_start_up=1,
       ramp_limit_shut_down=1,
       shut_down_cost=150,
       start_up_cost=200,
       p_nom=10000)

nu.add("Generator","gas",bus="bus",
       committable=True,
       marginal_cost=70,
       p_min_pu=0.1,
       up_time_before=2,
       min_up_time=3,
       shut_down_cost=20,
       start_up_cost=50,
       p_nom=1000)

nu.add("Load","load",bus="bus",p_set=p_set*sets_of_snapshots)

In [33]:
overlap = 2
for i in range(sets_of_snapshots):
    nu.lopf(nu.snapshots[i*len(p_set):(i+1)*len(p_set)+overlap])

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk


coal up time before: 1 , must stay up: 2
gas up time before: 2 , must stay up: 1
coal down time before: 0 , must stay down: 0


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 555370.0
  Upper bound: 555370.0
  Number of objectives: 1
  Number of constraints: 101
  Number of variables: 64
  Number of nonzeros: 248
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.020084381103515625
# ----------------------------------------------------------
#   Solution Information
# -------------------------

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation


...

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 550420.0
  Upper bound: 550420.0
  Number of objectives: 1
  Number of constraints: 103
  Number of variables: 64
  Number of nonzeros: 253
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.01661086082458496
# ----------------------------------------------------------
#   Solution Information
# --------------------------

INFO:pypsa.pf:Slack bus for sub-network 0 is bus
INFO:pypsa.opf:Performed preliminary steps
INFO:pypsa.opf:Building pyomo model using `angles` formulation
INFO:pypsa.opf:Solving model using glpk


coal up time before: 1 , must stay up: 2
gas up time before: 0 , must stay up: 0
coal down time before: 0 , must stay down: 0


INFO:pypsa.opf:Optimization successful


# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 370400.0
  Upper bound: 370400.0
  Number of objectives: 1
  Number of constraints: 73
  Number of variables: 46
  Number of nonzeros: 173
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 1
      Number of created subproblems: 1
  Error rc: 0
  Time: 0.018538713455200195
# ----------------------------------------------------------
#   Solution Information
# --------------------------

In [34]:
nu.generators_t.status

,coal,gas
0,1.0,1.0
1,1.0,1.0
2,0.0,1.0
3,0.0,1.0
4,1.0,0.0
5,1.0,0.0
6,1.0,1.0
7,0.0,1.0
8,0.0,1.0
9,1.0,0.0


In [35]:
nu.generators_t.p

,coal,gas
0,3900.0,100.0
1,4900.0,100.0
2,-0.0,700.0
3,-0.0,800.0
4,4000.0,-0.0
5,4000.0,-0.0
6,4900.0,100.0
7,-0.0,700.0
8,-0.0,800.0
9,4000.0,-0.0
